# Finetune ruGPT3Small on essays

## Install enviroment

In [ ]:
!pip3 install urllib3==1.25.4

     |████████████████████████████████| 133kB 8.0MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
!pip3 install transformers==2.8.0

     |████████████████████████████████| 573kB 14.4MB/s 
     |████████████████████████████████| 3.7MB 50.1MB/s 
     |████████████████████████████████| 133kB 50.9MB/s 
     |████████████████████████████████| 1.1MB 52.4MB/s 
     |████████████████████████████████| 890kB 38.0MB/s 
     |████████████████████████████████| 6.7MB 45.6MB/s 
     |████████████████████████████████| 71kB 10.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=d2af60a9ef17fc075d2c757d76aa1578b0f86f454c3902405f943b08d85031a3
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py

--2020-11-02 02:30:58--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/pretrain_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34150 (33K) [text/plain]
Saving to: ‘pretrain_transformers.py’

pretrain_transforme 100%[===================>]  33.35K  --.-KB/s    in 0.001s  

2020-11-02 02:30:58 (22.2 MB/s) - ‘pretrain_transformers.py’ saved [34150/34150]



In [ ]:
!wget https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py

--2020-11-02 02:30:58--  https://raw.githubusercontent.com/sberbank-ai/ru-gpts/master/generate_transformers.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10474 (10K) [text/plain]
Saving to: ‘generate_transformers.py’

generate_transforme 100%[===================>]  10.23K  --.-KB/s    in 0s      

2020-11-02 02:30:58 (104 MB/s) - ‘generate_transformers.py’ saved [10474/10474]



## Build Nvidia Apex from source
This allow us to use less GPU memory for finetuning

In [ ]:
!git clone https://github.com/NVIDIA/apex
!pip3 install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Cloning into 'apex'...
remote: Enumerating objects: 1, done.
remote: Counting objects: 100% (1/1), done.
remote: Total 7456 (delta 0), reused 0 (delta 0), pack-reused 7455
Receiving objects: 100% (7456/7456), 13.91 MiB | 26.48 MiB/s, done.
Resolving deltas: 100% (5036/5036), done.
/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-jbrpctme
Created temporary directory: /tmp/pip-req-tracker-jv75q6h5
Created requirements tracker '/tmp/pip-req-tracker-jv75q6h5'
Created temporary directory: /tmp/pip-install-mihto6kh
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-kfcq8f68
  Added file:///content/apex to build tracker '/tmp/pip-req-tracker-jv75q6h5'
    Running setup.py (path:/tmp/pip-req-build-kfcq8f68/setup.py) egg_info for pac

## Add data to colab
Add essays file from google dirve:
* Add [file](https://drive.google.com/file/d/10ZsjTeaoihYA80n1G40O5YZmaGw0yOXk/view?usp=sharing) to your own drive
* And mount drive to colab

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data_path = "/content/drive/My Drive/GPT-3 Colab/essays/essays.txt"
# data_path = "/content/drive/My Drive/GPT-3 Colab/stackoverflow/stackoverflow.txt"
!ls "$data_path"

'/content/drive/My Drive/GPT-3 Colab/essays/essays.txt'


## Prepare rus2eng data

In [ ]:
!wget https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11372/LRT-2610/paracrawl-release1.en-ru.zipporah0-dedup-clean.tgz?sequence=9&isAllowed=y
!mv 'paracrawl-release1.en-ru.zipporah0-dedup-clean.tgz?sequence=9' paracrawl-release1.en-ru.zipporah0-dedup-clean.tgz
!tar -zxvf paracrawl-release1.en-ru.zipporah0-dedup-clean.tgz

--2020-11-02 00:42:23--  https://lindat.mff.cuni.cz/repository/xmlui/bitstream/handle/11372/LRT-2610/paracrawl-release1.en-ru.zipporah0-dedup-clean.tgz?sequence=9
Resolving lindat.mff.cuni.cz (lindat.mff.cuni.cz)... 195.113.20.140
Connecting to lindat.mff.cuni.cz (lindat.mff.cuni.cz)|195.113.20.140|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 667981874 (637M) [application/x-gzip]
Saving to: ‘paracrawl-release1.en-ru.zipporah0-dedup-clean.tgz?sequence=9’

paracrawl-release1. 100%[===================>] 637.04M  27.3MB/s    in 27s     

2020-11-02 00:42:52 (23.2 MB/s) - ‘paracrawl-release1.en-ru.zipporah0-dedup-clean.tgz?sequence=9’ saved [667981874/667981874]



In [ ]:
!head -5 paracrawl-release1.en-ru.zipporah0-dedup-clean.en

E-mail: info@e-e-e.cz
The company has been registered with the Municipal Court in Prague, in Section B, File 14857.
News:
22.02.09
Since January 2009 new office premises in the street V Kolkovně 3, Prague 1, have been opened. These premises are situated in a close vicinity of all our partner banks.


In [ ]:
!head -5 paracrawl-release1.en-ru.zipporah0-dedup-clean.ru

Email: info@e-e-e.cz
Фирма зарегистрирована в Городском суде в г. Праге, раздел Б, вкладыш 14857.
Новости:
22.02.09
С января месяца 2009 г. открыты новые офисные помещения на улице V Kolkovně 3 (В Колковне, д. 3), Прага-1. Данные помещения находятся поблизости всех банков-партнеров.


In [ ]:
import re 

def len_cyrillic(text):
    return len(re.findall('[а-яА-Я]', text))

len_cyrillic('кек')

3

In [ ]:
all_translations = []
exmp_limit = 10000

with open('paracrawl-release1.en-ru.zipporah0-dedup-clean.en', 'r') as en_file, \
     open('paracrawl-release1.en-ru.zipporah0-dedup-clean.ru', 'r') as ru_file:
    c = 0
    while c < exmp_limit:
        ru_text = next(ru_file)[:-1]
        en_text = next(en_file)[:-1]
        if len(en_text) <= 150 and len_cyrillic(ru_text) > 10:
            all_translations.append(f'<s>Вопрос: {ru_text}\nОтвет: {en_text}</s>')
            c += 1

In [ ]:
print(all_translations[10])

<s>Вопрос: Наш опытный персонал всегда готов удовлетворить пожелания даже самого требовательного посетителя.
Ответ: Our experienced staff, is always available to serve the needs of even the most demanding customer.</s>


In [ ]:
import numpy as np
import random
random.seed(1234)
np.random.seed(1234)
np.random.shuffle(all_translations)
all_translations[1]

'<s>Вопрос: Жидкость в резервуаре – это накопленная энергия, которая регулярно сбрасывается и тем самым приводит в движение генератор.\nОтвет: The liquid inside the elevated tank becomes stored energy that is regularly drained in order to drive a turbine connected to a generator.</s>'

In [ ]:
boundary = int(exmp_limit*0.8)
train = all_translations[:boundary]
valid = all_translations[boundary:]
len(train), len(valid)

(8000, 2000)

In [ ]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

!rm gpt2_cached*

## Prepare essays data

In [ ]:
with open(data_path, "r") as file:
    text = file.read()

In [ ]:
valid_size = 5

In [ ]:
topics = []
all_essays = []
for line in text.split("</s>"):
    if "Тема:" in line and "Сочинение:" in line:
        essay_text = line.split("Сочинение:")
        if len(essay_text) == 2:
            topic = essay_text[0].replace("<s>", " ").replace("</s>", " ").strip()
            essay_text = essay_text[1].replace("<s>", " ").replace("</s>", " ").strip()
            essay_text = f"Сочинение: {essay_text}"
            essay_res = f"<s>{topic}\n{essay_text}</s>"
            all_essays.append(essay_res)
            topics.append(topic)

In [ ]:
import numpy as np
import random

In [ ]:
random.seed(1234)
np.random.seed(1234)

In [ ]:
unique_topics = list(set(topics))

In [ ]:
valid_topics = []

In [ ]:
for _ in range(valid_size):
    # Use randint for more speed (on big lists it is faster)
    idx = np.random.randint(0, len(unique_topics))
    valid_topics.append(unique_topics[idx])

In [ ]:
import nltk


train = []
valid = []
for topic, essay in zip(topics, all_essays):
    is_train = True
    for valid_topic in valid_topics:
        if (
            nltk.edit_distance(valid_topic, topic[:len(valid_topic)]) < 20 or
            nltk.edit_distance(valid_topic[:len(topic)], topic) < 20 or
            nltk.edit_distance(valid_topic[len(topic):], topic) < 20 or
            nltk.edit_distance(valid_topic, topic[len(valid_topic):]) < 20
            ):
            is_train = False
    if is_train:
        train.append(essay)
    else:
        valid.append(essay)

In [ ]:
len(train), len(valid)

(366, 34)

In [ ]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

## Prepare stackoverflow data

In [ ]:
with open(data_path, "r") as file:
    text = file.read()

In [ ]:
import re
text_filtered = text.replace('&lt;', '<').replace('&gt;', '>').replace('&#xA;', '\n').replace('&quot;', '"').replace('&nbsp;', ' ').replace('&amp;', '&').replace('&apos;', "'") #.replace('<EOS>\n','').split('<BOS>\n')[1]
text_filtered = re.sub('</?[a-z]+>', '', text_filtered)
text_filtered = re.sub('(\n\s*\n)+', '\n', text_filtered)
text_filtered = text_filtered.replace('<EOS>\n','').split('<BOS>\n')[1:]

In [ ]:
text_filtered[0]

'Q: Имеется:\nфайл file.txt в папке D:\\Test\nфайл file.txt в папке D:\\x64_this-is-my-test_54321_99.88.77.555_folder_n12345\nКак используя командную строку\n(FOR /R [[диск:]путь] %переменная IN (набор) DO команда [параметры]) \nCкопировать (xcopy /o /y) \nC заменой файл file.txt из папки D:\\Test в папку D:\\x64_this-is-my-test_54321_99.88.77.555_folder_n12345, если известно только что наименование второй папки начинается с x64_this-is-my-test, а дальнейший набор символов неизвестен?\nA: Я понял, что расположение второй папки известно, проблемы только с наименованием.\nЕсли это так, попробуйте использовать короткие имена файлов. Должно получится что-то в стиле  x64thi~ или  x64_th~.\nподробнее тут : <a href="https://support.microsoft.com/ru-ru/kb/142982/ru" rel="nofollow">https://support.microsoft.com/ru-ru/kb/142982/ru\nЕсли короткие имена не подходят, можно попробовать несколько извращенный вариант.\nfor /d %%f in ("x64_this-is-my-test*") do echo %%f\nв результате в переменную %%f б

In [ ]:
import numpy as np
import random

In [ ]:
random.seed(1234)
np.random.seed(1234)

In [ ]:
np.random.shuffle(text_filtered)
text_filtered[0]

'Q: Как известно, CSS-анимации создаются через @keyframes, и ключевые кадры там задаются в процентах. Так вот, как установить элементу конкретный процент, который должен отображаться? Сама анимация при этом должна быть приостановлена, просто должен отображаться выбранный скриптом кадр.\nЗачем это нужно: например, пользователь возюкает пальцем по экрану своего айфончика, и в зависимости от положения пальца нужно изменить положение и вид какого-нибудь элемента (свайп со сложной анимацией, например). Ну или произвольная анимация перелистывания слайдов в слайдере тем же пальцем.\nA: Проценты установить нельзя, но, если знать длительность анимации (animation-duration), можно пересчитать проценты в секунды и установить отрицательный animation-delay, так, чтобы анимация как бы начиналась с указанной секунды. Например, если длина анимации 3s и нам нужен кадр 50%, то ставим animation-delay: -1.5s (середина анимации).\nЧтобы это нормально работало, нужно соблюсти ещё несколько условий:\nanimatio

In [ ]:
all_QAs = []
exmp_limit = 500

for line in text_filtered[:exmp_limit]:
    if "Q: " in line and "\nA: " in line:
        essay_text = line.split("\nA: ")
        if len(essay_text) == 2:
            question = essay_text[0].strip()
            answer = essay_text[1].strip()
            answer = f"A: {answer}"
            QA = f"<s>{question}\n\n{answer}</s>"
            all_QAs.append(QA)

In [ ]:
boundary = int(exmp_limit*0.8)
train = all_QAs[:boundary]
valid = all_QAs[boundary:]
len(train), len(valid)

(400, 100)

In [ ]:
with open("train.txt", "w") as file:
    file.write("\n".join(train))

In [ ]:
with open("valid.txt", "w") as file:
    file.write("\n".join(valid))

## Run finetuning
The following code download our model and tokenizer from transformers and finetune model essays.

This took aroung ten minutes and obtain perplexity = tensor(13.8065)

In [ ]:
!python pretrain_transformers.py \
    --output_dir=translate_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt2large \
    --do_train \
    --train_data_file=train.txt \
    --do_eval \
    --eval_data_file=valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 1024 \
    --overwrite_output_dir \
    --fp16 \
    --fp16_opt_level=O2

2020-11-02 02:44:01.806876: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/02/2020 02:44:04 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: True
11/02/2020 02:44:04 - INFO - transformers.configuration_utils -   loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/sberbank-ai/rugpt2large/config.json from cache at /root/.cache/torch/transformers/ebe6992d0dca023ae6f51f91bf62ac9c23b25b6124d1ddd58b8bb2598a2da179.4b32e824394ba7bdd31d25b3d7541494f70fdb32554caf4178cd9ae8017976cc
11/02/2020 02:44:04 - INFO - transformers.configuration_utils -   Model config GPT2Config {
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": null,
  "attn_pdrop": 0.1,
  "bad_words_ids": null,
  "bos_token_id": 50256,
  "decoder_start_token_id": null,
  "do_sample": false,
  "early_stopping": false,
  "embd_pdrop": 0.1,
  "eos_to

## Check our model

In [ ]:
valid[0]

'<s>Вопрос: B.7 Если я путешествую с ребенком, надо ли мне заполнять отдельную анкету на ребенка?\nОтвет: Q.7 If I travel with a child, should I fill in a separate form for the child?</s>'

In [ ]:
!python generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=translate_model \
    --k=5 \
    --p=0.95 \
    --length=500

2020-11-02 02:08:26.148065: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
11/02/2020 02:08:27 - INFO - transformers.tokenization_utils -   Model name 'translate_model' not found in model shortcut name list (gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2). Assuming 'translate_model' is a path, a model identifier, or url to a directory containing tokenizer files.
11/02/2020 02:08:27 - INFO - transformers.tokenization_utils -   Didn't find file translate_model/added_tokens.json. We won't load it.
11/02/2020 02:08:27 - INFO - transformers.tokenization_utils -   loading file translate_model/vocab.json
11/02/2020 02:08:27 - INFO - transformers.tokenization_utils -   loading file translate_model/merges.txt
11/02/2020 02:08:27 - INFO - transformers.tokenization_utils -   loading file None
11/02/2020 02:08:27 - INFO - transformers.tokenization_utils -   loading file translate_model/special_tokens_map.json
11/02/2020 02:

In [ ]:
!ls essays_model

config.json	  merges.txt	     special_tokens_map.json  training_args.bin
eval_results.txt  pytorch_model.bin  tokenizer_config.json    vocab.json


In [ ]:
# !cp -r essays_model "./drive/My Drive/"

In [ ]:
!ls

sample_data
